In [1]:
import os
import re
import zlib
import json
import time
import random
import requests

In [13]:
with open('all_games.json', 'r', encoding='utf-8') as file:
    all_games = json.load(file)
''' 
with open('new_games_info.json', 'r', encoding='utf-8') as file:
    new_games = json.load(file)
with open('new_games_text.json', 'r', encoding='utf-8') as file:
    new_texts = json.load(file)
'''
with open('retro.json', 'r', encoding='utf-8') as file:
    retro = json.load(file)

In [ ]:
count = 1
for game_title, game in temp_dict.items():
    if 'igdb-cover' in game:
        try:
            images_path = 'static/Temp/' + game_title.replace(':', ' -').replace('/', ' ').replace('?', '') + '_Cover.jpg'
            response = requests.get(game['igdb-cover'])
            with open(images_path, 'wb') as img_file:
                img_file.write(response.content)
                print(count, game_title)
                count += 1
        except Exception as e:
            print(count, game_title, e)

In [ ]:
with open('all_games_cleaned.json', 'r', encoding='utf-8') as file:
    all_games = json.load(file)

'''
image_dict = dict()
for ch in '#ABCDEFGHIJKLMNOPQRSTUVWXYZ':
    for file in os.listdir('Images/Screenshots/' + ch + '/'):
        temp_file = file.split(' _')[0]
        if temp_file not in image_dict:
            image_dict[temp_file] = 0
        image_dict[temp_file] += 1
for file in os.listdir('Images/Retro Screenshots/'):
    temp_file = file.split(' _')[0]
    if temp_file not in image_dict:
        image_dict[temp_file] = 0
    image_dict[temp_file] += 1

to_download = set()
for game in all_games:
    temp_title = game.replace(':', ' -')
    if temp_title not in image_dict or image_dict[temp_title] < 10:
        to_download.add(game)
'''

for game, game_obj in all_games.items():
    ch = game[0]
    if ch not in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
        ch = '#'

    image_list = []
    for image_key in ['giantbomb-screenshots', 'igdb-screenshots', 'steam-images', 'riot-screenshots']:
        if image_key in game_obj:
            image_list.extend(game_obj[image_key])
    
    if 'gamesdb-images' in game_obj:
        for lst in game_obj['gamesdb-images'].values():
            if lst[0] == 'Screenshot - Gameplay':
                image_list.extend(lst[1:])

    if len(image_list) > 0:
        random.shuffle(image_list)
        count = 1
        for img in image_list[:10]:
            try:
                images_path = 'static/Temp/' + ch + '/' + game_obj['path'] + ' ' + str(count) + '.jpg'
                response = requests.get(img)
                with open(images_path, 'wb') as img_file:
                    img_file.write(response.content)
                    count += 1
            except Exception as _:
                pass
    print(game, count)

In [19]:
image_set = set()
for ch in '#ABCDEFGHIJKLMNOPQRSTUVWXYZ':
    for file in os.listdir('Images/Screenshots/' + ch + '/'):
        temp_file = file.split(' _')[0]
        if temp_file not in image_set:
            image_set.add(temp_file)
for file in os.listdir('Images/Retro Screenshots/'):
    temp_file = file.split(' _')[0]
    if temp_file not in image_set:
        image_set.add(temp_file)

to_download = set()
for game in all_games:
    temp_title = game.replace(':', ' -')
    if temp_title not in image_set:
        to_download.add(game)

In [15]:
for game_title in all_games:
    temp_title = game_title.replace(':', ' -').replace('/', ' ').replace('?', '')
    old_path = 'static/Covers New/' + temp_title + '_Cover.jpg'
    if os.path.exists(old_path):
        new_path = 'static/Covers New/' + all_games[game_title]['path'] + ' Cover.jpg'
        if old_path != new_path and not os.path.exists(new_path):
            os.rename(old_path, new_path)

In [ ]:
temp_dict = dict()
for game_title in all_games:
    temp_title = game_title.replace(':', ' -').replace('/', ' ').replace('?', '')
    if not os.path.exists('static/Covers/' + temp_title + '_Cover.jpg'):
        temp_dict[game_title] = all_games[game_title]
        print(game_title)
with open('to_update.json', 'w', encoding='utf-8') as file:
    json.dump(temp_dict, file)

In [ ]:
for game, game_obj in new_games.items():
    if game in all_games:
        all_games[game].update(game_obj)
    else:
        all_games[game] = game_obj

In [2]:
with open('new_games_2.json', 'r', encoding='utf-8') as file:
    new_games = json.load(file)

for game, game_obj in new_games.items():
    print(game, '#', game_obj.get('Release Date', [None])[0], '#',
          game_obj.get('metacritics-critics', '/'), '#',
          game_obj.get('metacritics-users', '/'), '#',
          game_obj.get('steam-positive', '/'), '#',
          game_obj.get('steam-nb-users', '/'), '#',
          game_obj.get('hltb-main', '/'), '#',
          game_obj.get('hltb-main+', '/'), '#',
          game_obj.get('hltb-complete', '/')
    )

Warhammer 40,000: Space Marine 2 # 2024-09-09 # 82 # 8.0 # 83 # 130753 # 9.62 # 15.37 # 68.86


In [5]:
with open('ordered_ultimate_games.json', 'r', encoding='utf-8') as file:
    or_ul_games = json.load(file)
with open('retro.json', 'r', encoding='utf-8') as file:
    retro = json.load(file)

In [ ]:
abbrev_set = set()
min_length = 6
for game in all_games:
    if len(game) < min_length:
        abbrev_set.add(game)
        print(game, '#', game, '#', game)
        all_games[game]['path'] = game
        continue
    letter_count = 2
    cleaned_game = re.sub(r'[^a-zA-Z0-9Ééöä]', ' ', game)
    cleaned_game = cleaned_game.strip()
    game_added = False
    while not game_added:
        game_abbr = ''
        for word in cleaned_game.split(' '):
            if word.isalpha():
                game_abbr += word[0:letter_count]
                continue
            if word.isnumeric() or word.isalnum() or word.replace(',', '').isnumeric():
                game_abbr += word
        if game_abbr in abbrev_set:
            letter_count += 1
        else:
            game_added = True
    abbrev_set.add(game_abbr)
    
    print(game, '#', cleaned_game, '#', game_abbr)
    all_games[game]['path'] = game_abbr
print(len(abbrev_set), len(all_games))

In [13]:
def generate_text(game_obj):
    text_obj = dict()
    str_obj = ''
    if game_obj.get('steam-description', '#') != '#':
        str_obj += 'Description: ' + game_obj['steam-description'] + '\n'
        text_obj['Intro'] = game_obj['steam-description']
    if game_obj.get('steam-summary', '#') != '#':
        str_obj += 'Summary: ' + game_obj['steam-summary'] + '\n'
    if str_obj not in ['', '#']:
        text_obj['steam-text'] = str_obj
    
    if game_obj.get('metacritics-description', '#') != '#':
        game_obj['metacritics-text'] = game_obj['metacritics-description']  + '\n'
        if 'Intro' not in text_obj:
            text_obj['Intro'] = game_obj['metacritics-text']

    if game_obj.get('rawg-description', '#') != '#':
        game_obj['rawg-text'] = game_obj['rawg-description'].replace('<p>', '').replace('</p>', '')  + '\n'
    
    str_obj = ''
    if game_obj.get('wikipedia-summary', '#') != '#':
        str_obj += 'Summary: ' + game_obj['wikipedia-summary'] + '\n'
        if 'Intro' not in text_obj:
            text_obj['Intro'] = game_obj['wikipedia-summary']
    if game_obj.get('wikipedia-gameplay', '#') != '#':
        str_obj += 'Gameplay: ' + game_obj['wikipedia-gameplay'] + '\n'
    if game_obj.get('wikipedia-plot', '#') != '#':
        str_obj += 'Plot: ' + game_obj['wikipedia-plot'] + '\n'
    if game_obj.get('wikipedia-synopsis', '#') != '#':
        str_obj += 'Synopsis: ' + game_obj['wikipedia-synopsis'] + '\n'
    if str_obj not in ['', '#']:
        text_obj['wikipedia-text'] = str_obj

    str_obj = ''
    if game_obj.get('giantbomb-intro', '#') != '#':
        str_obj += 'Intro: ' + game_obj['giantbomb-intro'] + '\n'
        if 'Intro' not in text_obj:
            text_obj['Intro'] = game_obj['giantbomb-intro']
    if game_obj.get('giantbomb-description', '#') != '#':
        str_obj += 'Description: ' + game_obj['giantbomb-description'] + '\n'
    if str_obj not in ['', '#']:
        text_obj['giantbomb-text'] = str_obj

    str_obj = ''
    if game_obj.get('igdb-summary', '#') != '#':
        str_obj += 'Summary: ' + game_obj['igdb-summary'] + '\n'
    if game_obj.get('igdb-story', '#') != '#':
        str_obj += 'Story: ' + game_obj['igdb-story'] + '\n'
    if str_obj not in ['', '#']:
        text_obj['igdb-text'] = str_obj

    if game_obj.get('backloggd-description', '#') != '#':
        text_obj['backloggd-text'] = game_obj['backloggd-description']  + '\n'

    if game_obj.get('moby-description', '#') != '#':
        text_obj['moby-text'] = game_obj['moby-description']  + '\n'

    if game_obj.get('gamesdb-description', '#') != '#':
        text_obj['gamesdb-text'] = game_obj['gamesdb-description']  + '\n'
    
    return text_obj

In [ ]:
with open('all_games.json', 'r', encoding='utf-8') as file:
    all_games = json.load(file)


def get_score_color(score, score_base):
    try:
        if score >= (score_base*0.8):
            return 'darkgreen'
        if score >= (score_base*0.7):
            return 'green'
        if score >= (score_base*0.6):
            return 'yellow'
        if score >= (score_base*0.5):
            return 'orange'
        if score >= (score_base*0.4):
            return 'red'
    except Exception as _:
        pass
    return 'purple'
    

cleaned_games = dict()
for game_title, game_obj in all_games.items():
    new_obj = dict()
    
    # Developers
    developers = set() 
    publishers = set()
    genres = set()
    similar_games = []
    release_date = []
    new_obj['sites'] = []
    scores = dict()

    base_scores = {'metacritics-critics': 10, 'metacritics-users': 100, 'steam-positive': 100, 'steam-nb-users':500, 
                   'backloggd-rating': 5, 'igdb-rating': 100, 'moby-internal-score': 10}

    for key, val in game_obj.items():
        # if any(x in key for x in ['summary', 'description', 'intro', 'plot', 'story', 'gameplay', 'synopsis']):
        #     continue

        if key in ['metacritics-critics', 'metacritics-users', 'steam-positive', 'steam-nb-users', 
            'backloggd-rating', 'igdb-rating', 'moby-internal-score']:
            scores[key.split('-').title()] = get_score_color(val, base_scores[key])
            continue

        if key in ['hltb-main', 'hltb-main+', 'hltb-complete']:
            scores[key.split('-').title()] = val
            continue

        if 'release-date' in key:
            release_date.append(val)
            continue

        if '-developers' in key:
            developers.update(set(val.split('; ')))
            continue
        if '-publishers' in key:
            publishers.update(set(val.split('; ')))
            continue
        if '-genres' in key and 'moby' not in key:
            genres.update(val.split('; '))
            continue
        if 'similar-titles' in key or 'similar_games' in key:
            for title in val.split('; '):
                if title in all_games:
                    similar_games.append(title)
            continue

        if 'riot' not in key and '-success' in key and val:
            new_obj['sites'].append(key.replace('-success', '').title())
            continue
        
        if val in ['', '#']:
            continue
        new_obj[key] = val
    
    if len(developers) > 0:
        new_obj['Developers'] = list(developers)
    if len(publishers) > 0:
        new_obj['Publishers'] = list(publishers)
    if len(genres) > 0:
        new_obj['Genres'] = list(genres)
    if len(similar_games) > 0:
        new_obj['Similar Games'] = similar_games
    if len(release_date) > 0:
        new_obj['Release Date'] = release_date
    
    # new_obj.update(generate_text(game_obj))
    cleaned_games[game_title] = new_obj

In [ ]:
# with open('all_games.json', 'w', encoding='utf-8') as file:
#     json.dump(all_games, file)

# with open('all_games_cleaned.json', 'w', encoding='utf-8') as file:
#     json.dump(cleaned_games, file)

In [2]:
from util.utility import get_raw_collection, get_soup

In [3]:
soup_giant = get_soup('https://www.giantbomb.com/hollow-knight/3030-48412/')
soup_wiki = get_soup('https://en.wikipedia.org/wiki/Hollow_Knight')
soup_moby = get_soup('https://www.mobygames.com/game/84194/hollow-knight/', cloud_scrapper=True)
soup_steam = get_soup('https://store.steampowered.com/app/367520/Hollow_Knight/', steam=True)

In [29]:
test = zlib.compress(str(soup_giant).encode('utf8'))
de_test = zlib.decompress(test)

In [48]:
de_test.decode('utf-8')

'<!DOCTYPE html>\n\n<html class="no-js no-touch" lang="en">\n<head><script type="text/javascript">(window.NREUM||(NREUM={})).init={privacy:{cookies_enabled:true},ajax:{deny_list:["bam.nr-data.net"]},distributed_tracing:{enabled:true}};(window.NREUM||(NREUM={})).loader_config={agentID:"1588750645",accountID:"3680512",trustKey:"3680512",xpid:"VwAPUVNSChABUVBUBAUAXlQH",licenseKey:"NRJS-433baf257505ff5076b",applicationID:"1516347806"};;/*! For license information please see nr-loader-spa-1.291.1.min.js.LICENSE.txt */\n(()=>{var e,t,r={8122:(e,t,r)=>{"use strict";r.d(t,{a:()=>i});var n=r(944);function i(e,t){try{if(!e||"object"!=typeof e)return(0,n.R)(3);if(!t||"object"!=typeof t)return(0,n.R)(4);const r=Object.create(Object.getPrototypeOf(t),Object.getOwnPropertyDescriptors(t)),o=0===Object.keys(r).length?e:r;for(let a in o)if(void 0!==e[a])try{if(null===e[a]){r[a]=null;continue}Array.isArray(e[a])&&Array.isArray(t[a])?r[a]=Array.from(new Set([...e[a],...t[a]])):"object"==typeof e[a]&&"obj

In [4]:
get_raw_collection().insert_one({'title': 'Hollow Knight', 'giant': zlib.compress(str(soup_giant).encode('utf8')),
                                 'steam': zlib.compress(str(soup_steam).encode('utf8')),
                                 'wikipedia': zlib.compress(str(soup_wiki).encode('utf8')),
                                 'moby': zlib.compress(str(soup_moby).encode('utf8'))})

InsertOneResult(ObjectId('684eb21175674a32547ad142'), acknowledged=True)